In [2]:
pip install librosa

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install mido

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install pyfluidsynth

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install midi2audio

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install pretty_midi

  Created wheel for pretty-midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592303 sha256=588c91ee8efd336cb56f5fdd1d98785fe0535bf56d1035cb0192dd75f8aae23b
  Stored in directory: c:\users\yimen\appdata\local\pip\cache\wheels\96\4a\bd\ea2d2c62f684a0bcce6e6d277cb69e4463b5fd372c422c2481
Successfully built pretty-midi
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install pydub

Note: you may need to restart the kernel to use updated packages.


# plot spectrogram

In [5]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

def plot_cqt(song, path):
  plt.figure(figsize=(7.5, 3.75))
  y, sr = librosa.load(song)
  C = librosa.cqt(y, sr=sr)
  librosa.display.specshow(librosa.amplitude_to_db(C, ref=np.max),
                            sr=sr)
  plt.axis('off')
  plt.savefig(path, bbox_inches="tight")
  plt.close('all')


In [6]:
def stft(sig, frameSize, overlapFac=0.5, window=np.hanning):
    win = window(frameSize)
    hopSize = int(frameSize - np.floor(overlapFac * frameSize))
    
    # zeros at beginning (thus center of 1st window should be for sample nr. 0)
    samples = np.append(np.zeros(np.int64(np.floor(frameSize/2.0))), sig)    
    # cols for windowing
    cols = np.int64(np.ceil( (len(samples) - frameSize) / float(hopSize)) + 1)
    # zeros at end (thus samples can be fully covered by frames)
    samples = np.append(samples, np.zeros(frameSize))
    
    frames = stride_tricks.as_strided(
            samples, shape=(cols, frameSize),
            strides=(samples.strides[0]*hopSize, samples.strides[0])).copy()
    frames *= win
    
    return np.fft.rfft(frames)  

""" scale frequency axis logarithmically """    
def logscale_spec(spec, sr=44100, factor=20.):
    timebins, freqbins = np.shape(spec)

    scale = np.linspace(0, 1, freqbins) ** factor
    scale *= (freqbins-1)/max(scale)
    scale = np.unique(np.round(scale))
    scale = np.int64(scale)
    
    # create spectrogram with new freq bins
    newspec = np.complex128(np.zeros([timebins, len(scale)]))
    for i in range(0, len(scale)):
        if i == len(scale)-1:
            newspec[:,i] = np.sum(spec[:,scale[i]:], axis=1)
        else:        
            newspec[:,i] = np.sum(spec[:,scale[i]:scale[i+1]], axis=1)
    
    # list center freq of bins
    allfreqs = np.abs(np.fft.fftfreq(freqbins*2, 1./sr)[:freqbins+1])
    freqs = []
    for i in range(0, len(scale)):
        if i == len(scale)-1:
            freqs += [np.mean(allfreqs[scale[i]:])]
        else:
            freqs += [np.mean(allfreqs[scale[i]:scale[i+1]])]
    
    return newspec, freqs

""" plot spectrogram"""
def plotstft(samples, samplerate, binsize=2**10, 
        plotpath=None, colormap="jet", plot_artifacts=True):
    s = stft(samples, binsize)
    
    sshow, freq = logscale_spec(s, factor=1.0, sr=samplerate)
    ims = 20.*np.log10(np.abs(sshow)/10e-6) # amplitude to decibel
    
    timebins, freqbins = np.shape(ims)
    
    plt.figure(figsize=(15, 7.5))
    plt.imshow(np.transpose(ims), origin="lower", 
            aspect="auto", cmap=colormap, interpolation="none")

    if not plot_artifacts:
      plt.axis('off')
    else:
      plt.colorbar()

      plt.xlabel("time (s)")
      plt.ylabel("frequency (hz)")
      plt.xlim([0, timebins-1])
      plt.ylim([0, freqbins])

      xlocs = np.float32(np.linspace(0, timebins-1, 5))
      plt.xticks(xlocs, ["%.02f" % l for l in 
          ((xlocs*len(samples)/timebins)+(0.5*binsize))/samplerate])

      ylocs = np.int16(np.round(np.linspace(0, freqbins-1, 10)))
      plt.yticks(ylocs, ["%.02f" % freq[i] for i in ylocs])
    
    if plotpath:
        plt.savefig(plotpath, bbox_inches="tight")
    else:
        plt.show()
    plt.clf()
    plt.close('all')

# SPLIT MIDI FILES

In [46]:
import mido
from mido import MidiFile, MidiTrack, Message, MetaMessage
from os import listdir
from os.path import isfile, split, join
import argparse

def split_midi(mid_file, target_dir, default_tempo=500000, target_segment_len=1/3):
  '''Split midi file into many chunks'''
  song_name = split(mid_file)[-1][:-4]
  mid = MidiFile(mid_file)
  time_split_arr = [0]

  # identify the meta messages
  metas = []
  tempo = default_tempo
  for msg in mid:
    if msg.type == 'set_tempo':
      tempo = msg.tempo
    if msg.is_meta:
      metas.append(msg)
  for meta in metas:
    meta.time = int(mido.second2tick(meta.time, mid.ticks_per_beat, tempo))

  target = MidiFile()
  track = MidiTrack()
  track.extend(metas)
  target.tracks.append(track)
  prefix = 0
  time_elapsed = 0
  total_time = 0
  for msg in mid:
    # Skip non-note related messages
    if msg.is_meta:
      continue
    time_elapsed += msg.time
    total_time += msg.time
    if msg.type != 'end_of_track':
      msg.time = int(mido.second2tick(msg.time, mid.ticks_per_beat, tempo))
      track.append(msg)
    if msg.type == 'end_of_track' or time_elapsed >= target_segment_len:
      track.append(MetaMessage('end_of_track'))
      time_split_arr.append(total_time)
      target.save(join(target_dir, song_name + '_{}.mid'.format(prefix)))
      target = MidiFile()
      track = MidiTrack()
      track.extend(metas)
      target.tracks.append(track)
      time_elapsed = 0
      prefix += 1
  return time_split_arr

        
def merge_midi(midis, input_dir, output, default_tempo=500000):
  '''Merge midi files into one'''
  pairs = [(int(x[:-4].split('_')[-1]), x) for x in midis]
  pairs = sorted(pairs, key=lambda x: x[0])
  midis = [join(input_dir, x[1]) for x in pairs]
#   print(midis)

  mid = MidiFile(midis[0])
  # identify the meta messages
  metas = [0]
  # tempo = default_tempo
  tempo = default_tempo // 2
  for msg in mid:
#     print(msg)
    if msg.type == 'set_tempo':
      print(msg.tempo)
      tempo = msg.tempo
    if msg.is_meta:
      metas.append(msg)
  for meta in metas:
    meta.time = int(mido.second2tick(meta.time, mid.ticks_per_beat, tempo))
  
  target = MidiFile()
  track = MidiTrack()
  track.extend(metas)
  target.tracks.append(track)
  for midi in midis:
    mid = MidiFile(midi)
    for msg in mid:
      if msg.is_meta:
        continue
      if msg.type != 'end_of_track':
        msg.time = int(mido.second2tick(msg.time, mid.ticks_per_beat, tempo))
        track.append(msg)

  track.append(MetaMessage('end_of_track'))
  target.save(output)


def parse_args():
  '''Parse arguments'''
  parser = argparse.ArgumentParser()
  parser.add_argument('input_dir', type=str)
  parser.add_argument('target_dir', type=str)
  parser.add_argument('-l', '--length', default=1, type=float)
  parser.add_argument('-m', '--merge', action='store_true')
  return parser.parse_args()


def split_midi_files(input_midi_dir, input_wav_dir, target_dir, length, merge=False):
#   args = parse_args()
#   input_dir = args.input_dir
#   target_dir = args.target_dir
#   length = args.length

  # Get all the input midi files
  midis = [x for x in listdir(input_midi_dir) if x.endswith('.mid')]

  if merge:
    merge_midi(midis, input_midi_dir, target_dir)
  else:
    for midi in midis:
      print(midi)
      try:
        time_split_arr = split_midi(join(input_midi_dir, midi), target_dir, target_segment_len=length)
        split_wav(input_wav_dir + '/' + midi[:-4] + '.wav', 'split_wav_based_on_midi/', time_split_arr)
      except:
        print('\tProblem!')

In [48]:
split_midi_files('sample_midi_files/', 'sample_wav_files', 'split_midi_files_now', 1/4)

en001a.mid
Splitting complete.
en001b.mid
Splitting complete.
en002a.mid
Splitting complete.
en002b.mid
Splitting complete.
en003a.mid
Splitting complete.
en003b.mid
Splitting complete.
en004a.mid
Splitting complete.
en004b.mid
Splitting complete.
en005a.mid
Splitting complete.
en005b.mid
Splitting complete.
en006a.mid
Splitting complete.
en006b.mid
Splitting complete.
en007a.mid
Splitting complete.
en007b.mid
Splitting complete.
en008a.mid
Splitting complete.
en008b.mid
Splitting complete.
en009a.mid
Splitting complete.
en009b.mid
Splitting complete.
en010a.mid
Splitting complete.
en010b.mid
Splitting complete.
en011a.mid
Splitting complete.
en011b.mid
Splitting complete.
en012a.mid
Splitting complete.
en012b.mid
Splitting complete.
en013a.mid
Splitting complete.
en013b.mid
Splitting complete.
en014a.mid
Splitting complete.
en014b.mid
Splitting complete.
en015a.mid
Splitting complete.
en015b.mid
Splitting complete.
en016a.mid
Splitting complete.
en016b.mid
Splitting complete.
en017a.m

In [95]:
testing = MidiFile('split_midi_files/en001a_5.mid')
for msg in testing: 
    print(msg)

MetaMessage('track_name', name='MIDI', time=0)
MetaMessage('set_tempo', tempo=600000, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
note_off channel=0 note=66 velocity=100 time=2.77625
note_on channel=0 note=66 velocity=100 time=0.3375
note_off channel=0 note=66 velocity=100 time=0.8625
MetaMessage('end_of_track', time=0)


# split wav file according to midi timing 

In [42]:
import wave

def split_wav(input_file, output_folder, timings):
    with wave.open(input_file, 'rb') as wav_file:
        frame_rate = wav_file.getframerate()
        sample_width = wav_file.getsampwidth()
        num_channels = wav_file.getnchannels()
        num_frames = wav_file.getnframes()
        params = wav_file.getparams()

        for i in range(len(timings) - 1):
            start_frame = int(timings[i] * frame_rate)
            end_frame = int(timings[i + 1] * frame_rate)

            wav_file.setpos(start_frame)
            frames = wav_file.readframes(end_frame - start_frame)
            
            song_name = input_file.split('/')[-1][:-4]
            output_file_path = f"{output_folder}{song_name}_{i}.wav"
            with wave.open(output_file_path, 'wb') as output_wav:
                output_wav.setparams(params)
                output_wav.writeframes(frames)

    print("Splitting complete.")

# Load your WAV file
input_file_path = 'sample_wav_files/en001a.wav'
output_folder = 'split_wav_based_on_midi/'
timings = [0, 2.4, 3.5812500000000003, 4.612500000000001, 5.793750000000001, 7.143750000000001, 8.2125, 9.262500000000001, 10.500000000000002, 11.925000000000002, 12.975000000000001, 14.381250000000001, 15.431250000000002, 16.8, 17.962500000000002, 19.143750000000004, 20.343750000000004, 21.600000000000005, 22.781250000000007, 23.868750000000006, 25.125000000000007, 26.343750000000007, 27.48750000000001, 28.762500000000014, 29.943750000000012, 36.05625000000001, 37.18125000000001, 38.28750000000001, 39.35625, 40.81875, 41.925, 43.05, 44.137499999999996, 45.58125, 46.65, 48.0, 49.06875, 50.400000000000006, 51.45, 52.78125, 53.9625, 55.2, 56.400000000000006, 57.46875000000001, 58.76250000000001, 60.000000000000014, 61.162500000000016, 62.40000000000001, 63.562500000000014]




In [46]:
split_wav(input_file_path, output_folder, timings)

FileNotFoundError: [Errno 2] No such file or directory: 'split_wav_based_on_midi/sample_wav_files\\en001a_0.wav'

# CONVERT WAV TO SPECTOGRAM

In [20]:
import sys
from os import listdir
from os.path import join, isfile
from subprocess import call
from scipy import signal
from scipy.io import wavfile
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
# import audiospec
import argparse
from midi2audio import FluidSynth
import numpy as np
from scipy import signal
import scipy.io.wavfile as wav
from numpy.lib import stride_tricks


def midi_to_wav(midi_file, output, soundfont='C:/Users/yimen/Documents/GitHub/converting-acapella-songs-to-sheet-music/FluidR3_GM.sf2'):
  '''Convert midi file on disk to wav'''
#   print(f"Soundfont path: {soundfont}")
#   command = ['fluidsynth', '-a', 'alsa', '-F', output, soundfont, midi_file]
  fs = FluidSynth(soundfont)
  fs.midi_to_audio(midi_file, output)
  return "done"


def wav_to_spectrogram(wav_file, output, segment=None):
  '''Convert wav file to spectogram'''
  # rate, data = wavfile.read(wav_file)
  # if segment is not None:
  #   data = data[segment[0] * rate:segment[1] * rate]
  # audiospec.plotstft(data[:, 0], rate, plotpath=output, plot_artifacts=False)
  plot_cqt(wav_file, output)

    
def wav_to_spectrogram_stft(wav_file, output, segment=None):
  '''Convert wav file to spectogram'''
  rate, data = wavfile.read(wav_file)
  if segment is not None:
    data = data[segment[0] * rate:segment[1] * rate]
  plotstft(data[:], rate, plotpath=output, plot_artifacts=False)
#   plot_cqt(wav_file, output)

In [22]:
def parse_args():
  '''Parse arguments'''
  parser = argparse.ArgumentParser()
  parser.add_argument('input_dir', type=str)
  parser.add_argument('wav_dir', type=str)
  parser.add_argument('spec_dir', type=str)
  parser.add_argument('-s', '--soundfont', default='/usr/share/sounds/sf2/FluidR3_GM.sf2', type=str)
  return parser.parse_args()

# CONVERT MIDI FILE TO SPECTOGRAM

In [21]:
def convert_midi_files_to_spectogram(input_dir, wav_dir, spec_dir, soundfont='C:/Users/yimen/Documents/GitHub/converting-acapella-songs-to-sheet-music/FluidR3_GM.sf2'):
  '''Convert directory of midi files into wav and spectrogram files'''
#   args = parse_args()

#   midi_dir = args.input_dir
#   wav_dir = args.wav_dir
#   spec_dir = args.spec_dir
#   soundfont = args.soundfont

  # Get all the input midi files
  midis = [x for x in listdir(input_dir) if x.endswith('.mid')]
  # Get existing wav files
  wavs = [x for x in listdir(wav_dir) if x.endswith('.wav')]
  # Get existing spectrograms
  specs = [x for x in listdir(spec_dir) if x.endswith('.jpg')]
  
  for midi in midis:
    print(midi)
    name = midi[:-4]
    if name + '.wav' not in wavs:
      midi_to_wav(join(input_dir, midi), join(wav_dir, name + '.wav'), soundfont=soundfont)
    if name + '.jpg' not in specs:
      wav_to_spectrogram_stft(join(wav_dir, name + '.wav'), join(spec_dir, name + '.jpg'))
  print('Done!')

In [83]:
convert_midi_files_to_spectogram('split_midi_files', 'split_wav_files', 'stft_output_spectrogram')

en001a_0.mid
en001a_1.mid


<ipython-input-37-6076b732de78>:10: UserWarning: amplitude_to_db was called on complex input so phase information will be discarded. To suppress this warning, call amplitude_to_db(np.abs(S)) instead.
  librosa.display.specshow(librosa.amplitude_to_db(C, ref=np.max),


en001a_10.mid
en001a_11.mid
en001a_12.mid
en001a_13.mid
en001a_14.mid
en001a_15.mid
en001a_16.mid
en001a_17.mid
en001a_18.mid
en001a_19.mid
en001a_2.mid
en001a_20.mid
en001a_21.mid
en001a_22.mid
en001a_23.mid
en001a_24.mid
en001a_25.mid
en001a_26.mid
en001a_27.mid
en001a_28.mid
en001a_29.mid
en001a_3.mid
en001a_30.mid
en001a_31.mid
en001a_32.mid
en001a_33.mid
en001a_34.mid
en001a_35.mid
en001a_36.mid
en001a_37.mid
en001a_38.mid
en001a_39.mid
en001a_4.mid
en001a_40.mid
en001a_41.mid
en001a_42.mid
en001a_43.mid
en001a_44.mid
en001a_45.mid
en001a_46.mid
en001a_47.mid
en001a_5.mid
en001a_6.mid
en001a_7.mid
en001a_8.mid
en001a_9.mid
en001b_0.mid
en001b_1.mid
en001b_10.mid
en001b_11.mid
en001b_12.mid
en001b_13.mid
en001b_14.mid
en001b_15.mid
en001b_16.mid
en001b_17.mid
en001b_18.mid
en001b_19.mid
en001b_2.mid
en001b_20.mid
en001b_21.mid
en001b_22.mid
en001b_23.mid
en001b_24.mid
en001b_25.mid
en001b_26.mid
en001b_27.mid
en001b_28.mid
en001b_29.mid
en001b_3.mid
en001b_30.mid
en001b_31.mid
en00

en005b_3.mid
en005b_30.mid
en005b_31.mid
en005b_32.mid
en005b_33.mid
en005b_34.mid
en005b_35.mid
en005b_36.mid
en005b_37.mid
en005b_38.mid
en005b_39.mid
en005b_4.mid
en005b_40.mid
en005b_41.mid
en005b_42.mid
en005b_43.mid
en005b_44.mid
en005b_45.mid
en005b_5.mid
en005b_6.mid
en005b_7.mid
en005b_8.mid
en005b_9.mid
en006a_0.mid
en006a_1.mid
en006a_10.mid
en006a_11.mid
en006a_12.mid
en006a_13.mid
en006a_14.mid
en006a_15.mid
en006a_16.mid
en006a_17.mid
en006a_18.mid
en006a_19.mid
en006a_2.mid
en006a_20.mid
en006a_21.mid
en006a_22.mid
en006a_23.mid
en006a_24.mid
en006a_25.mid
en006a_26.mid
en006a_27.mid
en006a_28.mid
en006a_29.mid
en006a_3.mid
en006a_30.mid
en006a_31.mid
en006a_32.mid
en006a_33.mid
en006a_34.mid
en006a_35.mid
en006a_36.mid
en006a_37.mid
en006a_38.mid
en006a_39.mid
en006a_4.mid
en006a_40.mid
en006a_41.mid
en006a_42.mid
en006a_43.mid
en006a_44.mid
en006a_45.mid
en006a_46.mid
en006a_47.mid
en006a_48.mid
en006a_5.mid
en006a_6.mid
en006a_7.mid
en006a_8.mid
en006a_9.mid
en006b_0.

en010a_21.mid
en010a_22.mid
en010a_23.mid
en010a_24.mid
en010a_25.mid
en010a_26.mid
en010a_27.mid
en010a_28.mid
en010a_29.mid
en010a_3.mid
en010a_30.mid
en010a_31.mid
en010a_32.mid
en010a_33.mid
en010a_34.mid
en010a_35.mid
en010a_36.mid
en010a_37.mid
en010a_38.mid
en010a_39.mid
en010a_4.mid
en010a_40.mid
en010a_41.mid
en010a_42.mid
en010a_43.mid
en010a_44.mid
en010a_45.mid
en010a_46.mid
en010a_47.mid
en010a_48.mid
en010a_49.mid
en010a_5.mid
en010a_50.mid
en010a_51.mid
en010a_52.mid
en010a_53.mid
en010a_54.mid
en010a_55.mid
en010a_56.mid
en010a_57.mid
en010a_58.mid
en010a_59.mid
en010a_6.mid
en010a_60.mid
en010a_61.mid
en010a_62.mid
en010a_63.mid
en010a_64.mid
en010a_65.mid
en010a_66.mid
en010a_67.mid
en010a_68.mid
en010a_69.mid
en010a_7.mid
en010a_70.mid
en010a_71.mid
en010a_72.mid
en010a_73.mid
en010a_74.mid
en010a_75.mid
en010a_76.mid
en010a_77.mid
en010a_78.mid
en010a_79.mid
en010a_8.mid
en010a_80.mid
en010a_81.mid
en010a_82.mid
en010a_83.mid
en010a_84.mid
en010a_85.mid
en010a_86.mi

...

en029b_4.mid
en029b_40.mid
en029b_41.mid
en029b_5.mid
en029b_6.mid
en029b_7.mid
en029b_8.mid
en029b_9.mid
en030a_0.mid
en030a_1.mid
en030a_10.mid
en030a_11.mid
en030a_12.mid
en030a_13.mid
en030a_14.mid
en030a_15.mid
en030a_16.mid
en030a_17.mid
en030a_18.mid
en030a_19.mid
en030a_2.mid
en030a_20.mid
en030a_21.mid
en030a_22.mid
en030a_23.mid
en030a_24.mid
en030a_25.mid
en030a_26.mid
en030a_27.mid
en030a_28.mid
en030a_29.mid
en030a_3.mid
en030a_30.mid
en030a_31.mid
en030a_32.mid
en030a_33.mid
en030a_34.mid
en030a_35.mid
en030a_36.mid
en030a_37.mid
en030a_38.mid
en030a_39.mid
en030a_4.mid
en030a_40.mid
en030a_41.mid
en030a_42.mid
en030a_43.mid
en030a_44.mid
en030a_45.mid
en030a_46.mid
en030a_47.mid
en030a_48.mid
en030a_49.mid
en030a_5.mid
en030a_50.mid
en030a_51.mid
en030a_52.mid
en030a_53.mid
en030a_54.mid
en030a_6.mid
en030a_7.mid
en030a_8.mid
en030a_9.mid
en030b_0.mid
en030b_1.mid
en030b_10.mid
en030b_11.mid
en030b_12.mid
en030b_13.mid
en030b_14.mid
en030b_15.mid
en030b_16.mid
en030b_17.

en034a_47.mid
en034a_48.mid
en034a_5.mid
en034a_6.mid
en034a_7.mid
en034a_8.mid
en034a_9.mid
en034b_0.mid
en034b_1.mid
en034b_10.mid
en034b_11.mid
en034b_12.mid
en034b_13.mid
en034b_14.mid
en034b_15.mid
en034b_16.mid
en034b_17.mid
en034b_18.mid
en034b_19.mid
en034b_2.mid
en034b_20.mid
en034b_21.mid
en034b_22.mid
en034b_23.mid
en034b_24.mid
en034b_25.mid
en034b_26.mid
en034b_27.mid
en034b_28.mid
en034b_29.mid
en034b_3.mid
en034b_30.mid
en034b_31.mid
en034b_32.mid
en034b_33.mid
en034b_34.mid
en034b_35.mid
en034b_36.mid
en034b_37.mid
en034b_38.mid
en034b_39.mid
en034b_4.mid
en034b_40.mid
en034b_41.mid
en034b_42.mid
en034b_43.mid
en034b_44.mid
en034b_45.mid
en034b_46.mid
en034b_47.mid
en034b_48.mid
en034b_5.mid
en034b_6.mid
en034b_7.mid
en034b_8.mid
en034b_9.mid
en035a_0.mid
en035a_1.mid
en035a_10.mid
en035a_11.mid
en035a_12.mid
en035a_13.mid
en035a_14.mid
en035a_15.mid
en035a_16.mid
en035a_17.mid
en035a_18.mid
en035a_19.mid
en035a_2.mid
en035a_20.mid
en035a_21.mid
en035a_22.mid
en035a_23.

en039b_37.mid
en039b_38.mid
en039b_39.mid
en039b_4.mid
en039b_40.mid
en039b_41.mid
en039b_42.mid
en039b_43.mid
en039b_44.mid
en039b_45.mid
en039b_46.mid
en039b_47.mid
en039b_48.mid
en039b_49.mid
en039b_5.mid
en039b_50.mid
en039b_51.mid
en039b_52.mid
en039b_53.mid
en039b_54.mid
en039b_55.mid
en039b_56.mid
en039b_57.mid
en039b_58.mid
en039b_59.mid
en039b_6.mid
en039b_60.mid
en039b_61.mid
en039b_62.mid
en039b_63.mid
en039b_64.mid
en039b_65.mid
en039b_66.mid
en039b_67.mid
en039b_68.mid
en039b_69.mid
en039b_7.mid
en039b_70.mid
en039b_71.mid
en039b_72.mid
en039b_73.mid
en039b_74.mid
en039b_75.mid
en039b_76.mid
en039b_77.mid
en039b_78.mid
en039b_79.mid
en039b_8.mid
en039b_80.mid
en039b_81.mid
en039b_82.mid
en039b_83.mid
en039b_84.mid
en039b_85.mid
en039b_86.mid
en039b_87.mid
en039b_88.mid
en039b_89.mid
en039b_9.mid
en039b_90.mid
en039b_91.mid
en039b_92.mid
en039b_93.mid
en040a_0.mid
en040a_1.mid
en040a_10.mid
en040a_11.mid
en040a_12.mid
en040a_13.mid
en040a_14.mid
en040a_15.mid
en040a_16.mid


en045a_2.mid
en045a_20.mid
en045a_21.mid
en045a_22.mid
en045a_23.mid
en045a_24.mid
en045a_25.mid
en045a_26.mid
en045a_27.mid
en045a_28.mid
en045a_29.mid
en045a_3.mid
en045a_30.mid
en045a_31.mid
en045a_32.mid
en045a_33.mid
en045a_34.mid
en045a_35.mid
en045a_36.mid
en045a_37.mid
en045a_4.mid
en045a_5.mid
en045a_6.mid
en045a_7.mid
en045a_8.mid
en045a_9.mid
en045b_0.mid
en045b_1.mid
en045b_10.mid
en045b_11.mid
en045b_12.mid
en045b_13.mid
en045b_14.mid
en045b_15.mid
en045b_16.mid
en045b_17.mid
en045b_18.mid
en045b_19.mid
en045b_2.mid
en045b_20.mid
en045b_21.mid
en045b_22.mid
en045b_23.mid
en045b_24.mid
en045b_25.mid
en045b_26.mid
en045b_27.mid
en045b_28.mid
en045b_29.mid
en045b_3.mid
en045b_30.mid
en045b_31.mid
en045b_32.mid
en045b_33.mid
en045b_34.mid
en045b_35.mid
en045b_36.mid
en045b_37.mid
en045b_4.mid
en045b_5.mid
en045b_6.mid
en045b_7.mid
en045b_8.mid
en045b_9.mid
en046a_0.mid
en046a_1.mid
en046a_10.mid
en046a_11.mid
en046a_12.mid
en046a_13.mid
en046a_14.mid
en046a_15.mid
en046a_16.mi

en050a_14.mid
en050a_15.mid
en050a_16.mid
en050a_17.mid
en050a_18.mid
en050a_19.mid
en050a_2.mid
en050a_20.mid
en050a_21.mid
en050a_22.mid
en050a_23.mid
en050a_24.mid
en050a_25.mid
en050a_26.mid
en050a_27.mid
en050a_28.mid
en050a_29.mid
en050a_3.mid
en050a_30.mid
en050a_31.mid
en050a_32.mid
en050a_33.mid
en050a_34.mid
en050a_35.mid
en050a_36.mid
en050a_37.mid
en050a_38.mid
en050a_39.mid
en050a_4.mid
en050a_40.mid
en050a_41.mid
en050a_42.mid
en050a_43.mid
en050a_44.mid
en050a_45.mid
en050a_46.mid
en050a_47.mid
en050a_48.mid
en050a_49.mid
en050a_5.mid
en050a_50.mid
en050a_51.mid
en050a_52.mid
en050a_53.mid
en050a_54.mid
en050a_55.mid
en050a_56.mid
en050a_57.mid
en050a_58.mid
en050a_59.mid
en050a_6.mid
en050a_60.mid
en050a_61.mid
en050a_62.mid
en050a_63.mid
en050a_64.mid
en050a_65.mid
en050a_66.mid
en050a_67.mid
en050a_68.mid
en050a_69.mid
en050a_7.mid
en050a_70.mid
en050a_71.mid
en050a_72.mid
en050a_73.mid
en050a_8.mid
en050a_9.mid
en050b_0.mid
en050b_1.mid
en050b_10.mid
en050b_11.mid
en

# PRETTY MIDI TO ONE HOT 

In [53]:
import pretty_midi
import numpy as np
import sys
import argparse

def pretty_midi_to_one_hot(pm, fs=100):
    """Compute a one hot matrix of a pretty midi object
    Parameters
    ----------
    pm : pretty_midi.PrettyMIDI
        A pretty_midi.PrettyMIDI class instance describing
        the piano roll.
    fs : int
        Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    Returns
    -------
    one_hot : np.ndarray, shape=(128,times.shape[0])
        Piano roll of this instrument. 1 represents Note Ons,
        -1 represents Note offs, 0 represents constant/do-nothing
    """

    # Allocate a matrix of zeros - we will add in as we go
    one_hots = []

    if len(pm.instruments) < 1:
        return 0

    for instrument in pm.instruments:
        one_hot = np.zeros((128, int(fs*instrument.get_end_time())+1))
        for note in instrument.notes:
            # note on
            one_hot[note.pitch, int(note.start*fs)] = 1
            # print('note on',note.pitch, int(note.start*fs))
            # note off
            one_hot[note.pitch, int(note.end*fs)] = 0
            # print('note off',note.pitch, int(note.end*fs))
        one_hots.append(one_hot)

    one_hot = np.zeros((128, np.max([o.shape[1] for o in one_hots])))
    for o in one_hots:
        one_hot[:, :o.shape[1]] += o

    one_hot = np.clip(one_hot,-1,1)
    return one_hot

def one_hot_to_pretty_midi(one_hot, fs=100, program=1,bpm=120):
    '''Convert a Piano Roll array into a PrettyMidi object
     with a single instrument.
    Parameters
    ----------
    piano_roll : np.ndarray, shape=(128,time)
        Piano roll of one instrument
    fs : int
        Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    program : int
        The program number of the instrument.
    bpm : int
        Beats per minute, used to decide when to re-emphasize notes left on.
    Returns
    -------
    midi_object : pretty_midi.PrettyMIDI
        A pretty_midi.PrettyMIDI class instance describing
        the piano roll.
    '''
    notes, frames = one_hot.shape
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=program)

    # prepend, append zeros so we can acknowledge inital and ending events
    piano_roll = np.hstack((np.zeros((notes, 1)),
                            one_hot,
                            np.zeros((notes, 1))))

    # use changes to find note on / note off events
    changes = np.nonzero(np.diff(piano_roll).T)

    # keep track of note on times and notes currently playing
    note_on_time = np.zeros(notes)
    current_notes = np.zeros(notes)

    bps = bpm / 60
    beat_interval = fs / bps
    strong_beats = beat_interval * 2 #(for 4/4 timing)

    last_beat_time = 0

    for time, note in zip(*changes):
        change = piano_roll[note, time + 1]

        if time >= last_beat_time + beat_interval:
            for note in current_notes:
                time = time / fs

        time = time / fs
        if change == 1:
            # note on
            if current_notes[note] == 0:
                # from note off
                note_on_time[note] = time
                current_notes[note] = 1
            else:
                #re-articulate (later in code)
                '''pm_note = pretty_midi.Note(
                        velocity=100, #don't care fer now
                        pitch=note,
                        start=note_on_time[note],
                        end=time)
                instrument.notes.append(pm_note)
                note_on_time[note] = time
                current_notes[note] = 1'''
        elif change == 0:
            #note off
            pm_note = pretty_midi.Note(
                    velocity=100, #don't care fer now
                    pitch=note,
                    start=note_on_time[note],
                    end=time)
            current_notes[note] = 0
            instrument.notes.append(pm_note)
    pm.instruments.append(instrument)
    return pm

def slice_to_categories(piano_roll):
    notes_list = np.zeros(128)
    notes = np.nonzero(piano_roll)[0]
    notes = np.unique(notes)

    for note in notes:
        notes_list[note] = 1

    return notes_list


# CNN MODEL 

In [51]:
import numpy as np
import keras
from keras.layers import Dense, Flatten, Reshape, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from  skimage.measure import block_reduce

from PIL import Image
import pretty_midi
import os, os.path
import sys

def create_model():
    img_x, img_y = 145, 49
    input_shape = (img_x, img_y, 3)
    num_classes = 128

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5,5), strides=(1,1),
        activation='tanh',
        input_shape=input_shape))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Conv2D(64, (3,3), activation='tanh'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2,2)))
    #model.add(Conv2D(64, (5,5), activation='relu'))
    # Final output layer
    #model.add(Conv2D(128, (5,5), activation='sigmoid'))
    #model.add(Flatten())
    model.add(Flatten())
    #model.add(Dense(64, activation='sigmoid'))
    model.add(Dense(num_classes, activation='sigmoid'))
    return model
    

class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))


def train(x_train, y_train, x_test, y_test):
    batch_size = 64
    num_classes = 128
    epochs = 100

    # input image dimensions
    img_x, img_y = 624, 1222

    path = 'cnn_model'
    #path = '/mnt/c/Users/chau/Documents/models'
    model_ckpt = os.path.join(path,'newckpt.h5')
    
    #x_train = x_train.reshape(x_train.shape[0], img_x, img_y, 3)
    #x_test = x_test.reshape(x_train.shape[0], img_x, img_y, 3)
    
    # Convert data to right type
    #x_train = x_train.astype('float32')
    #x_test = x_test.astype('float32')
    #x_train /= 255
    #x_test /= 255
    #print(x_train)
    #print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    model = create_model()
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)
    model.compile(loss=keras.losses.binary_crossentropy,
            optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
            metrics=['accuracy'])

    history = AccuracyHistory()

    checkpoint = ModelCheckpoint(model_ckpt,
            monitor='val_loss',
            verbose=1,
            save_best_only=True,
            mode='min')
    early_stop = EarlyStopping(patience=5, 
            monitor='val_loss',
            verbose=1, mode='min')
    callbacks = [history, checkpoint,early_stop]
    
    model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            validation_data=(x_test, y_test),
            callbacks=callbacks)

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
#     plt.plot(range(1,101), history.acc)
#     plt.xlabel('Epochs')
#     plt.ylabel('Accuracy')
#     plt.savefig('loss.png')
#     plt.show()


def run_cnn(jpg_path, midi_path):
    # x is spectrogram, y is MIDI
    #jpg_path = '/mnt/d/Workspace/EE379K/data/spectrograms'
    #midi_path = '/mnt/d/Workspace/EE379K/data/split_midi'
    #jpg_path = '/mnt/c/Users/chau/Documents/spectrograms'
    #midi_path = '/mnt/c/Users/chau/Documents/split_midi'
    x_train, y_train = [], []
    img = []
    i = 0
    for filename in os.listdir(jpg_path):
        print(filename)
        # filename = "daylight_128.jpg"
        m_fn = filename.replace(".jpg", ".mid")
        if os.path.isfile(os.path.join(midi_path, m_fn)):
            pm = pretty_midi.PrettyMIDI(os.path.join(midi_path, m_fn))
            oh = pretty_midi_to_one_hot(pm)
            if type(oh) is not int:
                oh = slice_to_categories(oh)
                #oh = oh.reshape(1, 128)
                y_train.append(oh)
        
                im = Image.open(os.path.join(jpg_path, filename))
                im = im.crop((14, 13, 594, 301))
                resize = im.resize((49, 145), Image.NEAREST)
                resize.load()
                #result = Image.fromarray((visual * 255).astype(numpy.uint8))
                #resize.save("images/" + str(i) + ".jpg")
                arr = np.asarray(resize, dtype="float32")
                #print(arr)
                #arr = block_reduce(arr, block_size=(2,2,1), func=np.mean)
                x_train.append(arr)
                #if len(x_train) > 0:
                #    break
                i += 1

    x_train = np.array(x_train)
    #x_train = x_train.reshape(len(x_train), 1)
    y_train = np.array(y_train)
    #print(y_train)
    #print(x_train.shape)
    #print(y_train.shape)
    #print(len(x_train))
    #print(np.shape(x_train))
    #im_array = np.array([np.array
    #x_train = np.array(x_train)
    x_test = np.copy(x_train)
    y_test = np.copy(y_train)
    #x_train, x_test, y_train, y_test = train_test_split(
    #        x_train, y_train, test_size=0.2, random_state=1)
    #print(x_train.shape)
    #print(y_train.shape)
    x_train /= 255.0
    x_test /= 255.0
    train(x_train, y_train, x_test, y_test)


In [85]:
run_cnn('stft_output_spectrogram', 'split_midi_files') split_wav_spectrogram

en001a_0.jpg
en001a_1.jpg
en001a_10.jpg
en001a_11.jpg
en001a_12.jpg
en001a_13.jpg
en001a_14.jpg
en001a_15.jpg
en001a_16.jpg
en001a_17.jpg
en001a_18.jpg
en001a_19.jpg
en001a_2.jpg
en001a_20.jpg
en001a_21.jpg
en001a_22.jpg
en001a_23.jpg
en001a_24.jpg
en001a_25.jpg
en001a_26.jpg
en001a_27.jpg
en001a_28.jpg
en001a_29.jpg
en001a_3.jpg
en001a_30.jpg
en001a_31.jpg
en001a_32.jpg
en001a_33.jpg
en001a_34.jpg
en001a_35.jpg
en001a_36.jpg
en001a_37.jpg
en001a_38.jpg
en001a_39.jpg
en001a_4.jpg
en001a_40.jpg
en001a_41.jpg
en001a_42.jpg
en001a_43.jpg
en001a_44.jpg
en001a_45.jpg
en001a_46.jpg
en001a_47.jpg
en001a_5.jpg
en001a_6.jpg
en001a_7.jpg
en001a_8.jpg
en001a_9.jpg
en001b_0.jpg
en001b_1.jpg
en001b_10.jpg
en001b_11.jpg
en001b_12.jpg
en001b_13.jpg
en001b_14.jpg
en001b_15.jpg
en001b_16.jpg
en001b_17.jpg
en001b_18.jpg
en001b_19.jpg
en001b_2.jpg
en001b_20.jpg
en001b_21.jpg
en001b_22.jpg
en001b_23.jpg
en001b_24.jpg
en001b_25.jpg
en001b_26.jpg
en001b_27.jpg
en001b_28.jpg
en001b_29.jpg
en001b_3.jpg
en001b

en005b_33.jpg
en005b_34.jpg
en005b_35.jpg
en005b_36.jpg
en005b_37.jpg
en005b_38.jpg
en005b_39.jpg
en005b_4.jpg
en005b_40.jpg
en005b_41.jpg
en005b_42.jpg
en005b_43.jpg
en005b_44.jpg
en005b_45.jpg
en005b_5.jpg
en005b_6.jpg
en005b_7.jpg
en005b_8.jpg
en005b_9.jpg
en006a_0.jpg
en006a_1.jpg
en006a_10.jpg
en006a_11.jpg
en006a_12.jpg
en006a_13.jpg
en006a_14.jpg
en006a_15.jpg
en006a_16.jpg
en006a_17.jpg
en006a_18.jpg
en006a_19.jpg
en006a_2.jpg
en006a_20.jpg
en006a_21.jpg
en006a_22.jpg
en006a_23.jpg
en006a_24.jpg
en006a_25.jpg
en006a_26.jpg
en006a_27.jpg
en006a_28.jpg
en006a_29.jpg
en006a_3.jpg
en006a_30.jpg
en006a_31.jpg
en006a_32.jpg
en006a_33.jpg
en006a_34.jpg
en006a_35.jpg
en006a_36.jpg
en006a_37.jpg
en006a_38.jpg
en006a_39.jpg
en006a_4.jpg
en006a_40.jpg
en006a_41.jpg
en006a_42.jpg
en006a_43.jpg
en006a_44.jpg
en006a_45.jpg
en006a_46.jpg
en006a_47.jpg
en006a_48.jpg
en006a_5.jpg
en006a_6.jpg
en006a_7.jpg
en006a_8.jpg
en006a_9.jpg
en006b_0.jpg
en006b_1.jpg
en006b_10.jpg
en006b_11.jpg
en006b_12.

en010a_81.jpg
en010a_82.jpg
en010a_83.jpg
en010a_84.jpg
en010a_85.jpg
en010a_86.jpg
en010a_87.jpg
en010a_88.jpg
en010a_89.jpg
en010a_9.jpg
en010a_90.jpg
en010a_91.jpg
en010a_92.jpg
en010a_93.jpg
en010a_94.jpg
en010a_95.jpg
en010a_96.jpg
en010a_97.jpg
en010a_98.jpg
en010a_99.jpg
en010b_0.jpg
en010b_1.jpg
en010b_10.jpg
en010b_100.jpg
en010b_101.jpg
en010b_102.jpg
en010b_103.jpg
en010b_104.jpg
en010b_105.jpg
en010b_106.jpg
en010b_107.jpg
en010b_108.jpg
en010b_109.jpg
en010b_11.jpg
en010b_110.jpg
en010b_12.jpg
en010b_13.jpg
en010b_14.jpg
en010b_15.jpg
en010b_16.jpg
en010b_17.jpg
en010b_18.jpg
en010b_19.jpg
en010b_2.jpg
en010b_20.jpg
en010b_21.jpg
en010b_22.jpg
en010b_23.jpg
en010b_24.jpg
en010b_25.jpg
en010b_26.jpg
en010b_27.jpg
en010b_28.jpg
en010b_29.jpg
en010b_3.jpg
en010b_30.jpg
en010b_31.jpg
en010b_32.jpg
en010b_33.jpg
en010b_34.jpg
en010b_35.jpg
en010b_36.jpg
en010b_37.jpg
en010b_38.jpg
en010b_39.jpg
en010b_4.jpg
en010b_40.jpg
en010b_41.jpg
en010b_42.jpg
en010b_43.jpg
en010b_44.jpg
e

en013b_8.jpg
en013b_9.jpg
en014a_0.jpg
en014a_1.jpg
en014a_10.jpg
en014a_11.jpg
en014a_12.jpg
en014a_13.jpg
en014a_14.jpg
en014a_15.jpg
en014a_16.jpg
en014a_17.jpg
en014a_18.jpg
en014a_19.jpg
en014a_2.jpg
en014a_20.jpg
en014a_21.jpg
en014a_22.jpg
en014a_23.jpg
en014a_24.jpg
en014a_25.jpg
en014a_26.jpg
en014a_27.jpg
en014a_28.jpg
en014a_29.jpg
en014a_3.jpg
en014a_30.jpg
en014a_31.jpg
en014a_32.jpg
en014a_33.jpg
en014a_34.jpg
en014a_35.jpg
en014a_36.jpg
en014a_37.jpg
en014a_38.jpg
en014a_39.jpg
en014a_4.jpg
en014a_40.jpg
en014a_41.jpg
en014a_42.jpg
en014a_43.jpg
en014a_44.jpg
en014a_45.jpg
en014a_46.jpg
en014a_47.jpg
en014a_48.jpg
en014a_49.jpg
en014a_5.jpg
en014a_50.jpg
en014a_51.jpg
en014a_52.jpg
en014a_53.jpg
en014a_54.jpg
en014a_55.jpg
en014a_56.jpg
en014a_57.jpg
en014a_58.jpg
en014a_59.jpg
en014a_6.jpg
en014a_60.jpg
en014a_61.jpg
en014a_62.jpg
en014a_63.jpg
en014a_64.jpg
en014a_65.jpg
en014a_66.jpg
en014a_67.jpg
en014a_7.jpg
en014a_8.jpg
en014a_9.jpg
en014b_0.jpg
en014b_1.jpg
en014b

en020b_30.jpg
en020b_31.jpg
en020b_4.jpg
en020b_5.jpg
en020b_6.jpg
en020b_7.jpg
en020b_8.jpg
en020b_9.jpg
en021a_0.jpg
en021a_1.jpg
en021a_10.jpg
en021a_11.jpg
en021a_12.jpg
en021a_13.jpg
en021a_14.jpg
en021a_15.jpg
en021a_16.jpg
en021a_17.jpg
en021a_18.jpg
en021a_19.jpg
en021a_2.jpg
en021a_20.jpg
en021a_21.jpg
en021a_22.jpg
en021a_23.jpg
en021a_24.jpg
en021a_25.jpg
en021a_26.jpg
en021a_27.jpg
en021a_28.jpg
en021a_29.jpg
en021a_3.jpg
en021a_30.jpg
en021a_31.jpg
en021a_32.jpg
en021a_33.jpg
en021a_34.jpg
en021a_35.jpg
en021a_36.jpg
en021a_37.jpg
en021a_38.jpg
en021a_39.jpg
en021a_4.jpg
en021a_40.jpg
en021a_41.jpg
en021a_42.jpg
en021a_43.jpg
en021a_44.jpg
en021a_45.jpg
en021a_46.jpg
en021a_47.jpg
en021a_48.jpg
en021a_49.jpg
en021a_5.jpg
en021a_50.jpg
en021a_51.jpg
en021a_52.jpg
en021a_53.jpg
en021a_54.jpg
en021a_55.jpg
en021a_56.jpg
en021a_57.jpg
en021a_58.jpg
en021a_59.jpg
en021a_6.jpg
en021a_60.jpg
en021a_61.jpg
en021a_62.jpg
en021a_63.jpg
en021a_64.jpg
en021a_7.jpg
en021a_8.jpg
en021a_

...

en036a_11.jpg
en036a_12.jpg
en036a_13.jpg
en036a_14.jpg
en036a_15.jpg
en036a_16.jpg
en036a_17.jpg
en036a_18.jpg
en036a_19.jpg
en036a_2.jpg
en036a_20.jpg
en036a_21.jpg
en036a_22.jpg
en036a_23.jpg
en036a_24.jpg
en036a_25.jpg
en036a_26.jpg
en036a_27.jpg
en036a_28.jpg
en036a_29.jpg
en036a_3.jpg
en036a_30.jpg
en036a_31.jpg
en036a_32.jpg
en036a_33.jpg
en036a_34.jpg
en036a_35.jpg
en036a_36.jpg
en036a_37.jpg
en036a_4.jpg
en036a_5.jpg
en036a_6.jpg
en036a_7.jpg
en036a_8.jpg
en036a_9.jpg
en036b_0.jpg
en036b_1.jpg
en036b_10.jpg
en036b_11.jpg
en036b_12.jpg
en036b_13.jpg
en036b_14.jpg
en036b_15.jpg
en036b_16.jpg
en036b_17.jpg
en036b_18.jpg
en036b_19.jpg
en036b_2.jpg
en036b_20.jpg
en036b_21.jpg
en036b_22.jpg
en036b_23.jpg
en036b_24.jpg
en036b_25.jpg
en036b_26.jpg
en036b_27.jpg
en036b_28.jpg
en036b_29.jpg
en036b_3.jpg
en036b_30.jpg
en036b_31.jpg
en036b_32.jpg
en036b_33.jpg
en036b_34.jpg
en036b_35.jpg
en036b_36.jpg
en036b_37.jpg
en036b_4.jpg
en036b_5.jpg
en036b_6.jpg
en036b_7.jpg
en036b_8.jpg
en036b_9.

en041b_8.jpg
en041b_9.jpg
en042a_0.jpg
en042a_1.jpg
en042a_10.jpg
en042a_11.jpg
en042a_12.jpg
en042a_13.jpg
en042a_14.jpg
en042a_15.jpg
en042a_16.jpg
en042a_17.jpg
en042a_18.jpg
en042a_19.jpg
en042a_2.jpg
en042a_20.jpg
en042a_21.jpg
en042a_22.jpg
en042a_23.jpg
en042a_24.jpg
en042a_25.jpg
en042a_26.jpg
en042a_27.jpg
en042a_28.jpg
en042a_29.jpg
en042a_3.jpg
en042a_30.jpg
en042a_31.jpg
en042a_32.jpg
en042a_33.jpg
en042a_34.jpg
en042a_35.jpg
en042a_36.jpg
en042a_37.jpg
en042a_38.jpg
en042a_39.jpg
en042a_4.jpg
en042a_40.jpg
en042a_41.jpg
en042a_42.jpg
en042a_43.jpg
en042a_44.jpg
en042a_45.jpg
en042a_46.jpg
en042a_47.jpg
en042a_48.jpg
en042a_49.jpg
en042a_5.jpg
en042a_6.jpg
en042a_7.jpg
en042a_8.jpg
en042a_9.jpg
en042b_0.jpg
en042b_1.jpg
en042b_10.jpg
en042b_11.jpg
en042b_12.jpg
en042b_13.jpg
en042b_14.jpg
en042b_15.jpg
en042b_16.jpg
en042b_17.jpg
en042b_18.jpg
en042b_19.jpg
en042b_2.jpg
en042b_20.jpg
en042b_21.jpg
en042b_22.jpg
en042b_23.jpg
en042b_24.jpg
en042b_25.jpg
en042b_26.jpg
en042b_

en047b_45.jpg
en047b_46.jpg
en047b_47.jpg
en047b_48.jpg
en047b_49.jpg
en047b_5.jpg
en047b_50.jpg
en047b_51.jpg
en047b_52.jpg
en047b_53.jpg
en047b_54.jpg
en047b_55.jpg
en047b_56.jpg
en047b_57.jpg
en047b_58.jpg
en047b_59.jpg
en047b_6.jpg
en047b_60.jpg
en047b_61.jpg
en047b_62.jpg
en047b_63.jpg
en047b_7.jpg
en047b_8.jpg
en047b_9.jpg
en048a_0.jpg
en048a_1.jpg
en048a_10.jpg
en048a_11.jpg
en048a_12.jpg
en048a_13.jpg
en048a_14.jpg
en048a_15.jpg
en048a_16.jpg
en048a_17.jpg
en048a_18.jpg
en048a_19.jpg
en048a_2.jpg
en048a_20.jpg
en048a_21.jpg
en048a_22.jpg
en048a_23.jpg
en048a_24.jpg
en048a_25.jpg
en048a_26.jpg
en048a_27.jpg
en048a_28.jpg
en048a_29.jpg
en048a_3.jpg
en048a_30.jpg
en048a_31.jpg
en048a_32.jpg
en048a_33.jpg
en048a_34.jpg
en048a_35.jpg
en048a_36.jpg
en048a_37.jpg
en048a_38.jpg
en048a_39.jpg
en048a_4.jpg
en048a_40.jpg
en048a_41.jpg
en048a_42.jpg
en048a_43.jpg
en048a_44.jpg
en048a_45.jpg
en048a_46.jpg
en048a_47.jpg
en048a_48.jpg
en048a_49.jpg
en048a_5.jpg
en048a_50.jpg
en048a_51.jpg
en0

C:\Users\yimen\Anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


86/86 [==============================] - ETA: 0s - loss: 0.0381 - accuracy: 0.3787
Epoch 2: val_loss improved from 445.06821 to 438.15512, saving model to cnn_model\ckpt.h5
86/86 [==============================] - 37s 432ms/step - loss: 0.0381 - accuracy: 0.3787 - val_loss: 438.1551 - val_accuracy: 0.0000e+00
Epoch 3/100
86/86 [==============================] - ETA: 0s - loss: 0.0239 - accuracy: 0.5890
Epoch 3: val_loss improved from 438.15512 to 425.78049, saving model to cnn_model\ckpt.h5
86/86 [==============================] - 37s 434ms/step - loss: 0.0239 - accuracy: 0.5890 - val_loss: 425.7805 - val_accuracy: 0.0000e+00
Epoch 4/100
86/86 [==============================] - ETA: 0s - loss: 0.0150 - accuracy: 0.6579
Epoch 4: val_loss improved from 425.78049 to 414.58969, saving model to cnn_model\ckpt.h5
86/86 [==============================] - 38s 437ms/step - loss: 0.0150 - accuracy: 0.6579 - val_loss: 414.5897 - val_accuracy: 0.0000e+00
Epoch 5/100
86/86 [========================

In [54]:
# for training split wav spectrogram and the output midi
run_cnn('split_wav_based_on_midi_spectrogram', 'split_midi_files_now')

en001a_0.jpg
en001a_1.jpg
en001a_10.jpg
en001a_100.jpg
en001a_101.jpg
en001a_102.jpg
en001a_11.jpg
en001a_12.jpg
en001a_13.jpg
en001a_14.jpg
en001a_15.jpg
en001a_16.jpg
en001a_17.jpg
en001a_18.jpg
en001a_19.jpg
en001a_2.jpg
en001a_20.jpg
en001a_21.jpg
en001a_22.jpg
en001a_23.jpg
en001a_24.jpg
en001a_25.jpg
en001a_26.jpg
en001a_27.jpg
en001a_28.jpg
en001a_29.jpg
en001a_3.jpg
en001a_30.jpg
en001a_31.jpg
en001a_32.jpg
en001a_33.jpg
en001a_34.jpg
en001a_35.jpg
en001a_36.jpg
en001a_37.jpg
en001a_38.jpg
en001a_39.jpg
en001a_4.jpg
en001a_40.jpg
en001a_41.jpg
en001a_42.jpg
en001a_43.jpg
en001a_44.jpg
en001a_45.jpg
en001a_46.jpg
en001a_47.jpg
en001a_48.jpg
en001a_49.jpg
en001a_5.jpg
en001a_50.jpg
en001a_51.jpg
en001a_52.jpg
en001a_53.jpg
en001a_54.jpg
en001a_55.jpg
en001a_56.jpg
en001a_57.jpg
en001a_58.jpg
en001a_59.jpg
en001a_6.jpg
en001a_60.jpg
en001a_61.jpg
en001a_62.jpg
en001a_63.jpg
en001a_64.jpg
en001a_65.jpg
en001a_66.jpg
en001a_67.jpg
en001a_68.jpg
en001a_69.jpg
en001a_7.jpg
en001a_70.j

en002b_246.jpg
en002b_247.jpg
en002b_248.jpg
en002b_249.jpg
en002b_25.jpg
en002b_250.jpg
en002b_251.jpg
en002b_252.jpg
en002b_253.jpg
en002b_26.jpg
en002b_27.jpg
en002b_28.jpg
en002b_29.jpg
en002b_3.jpg
en002b_30.jpg
en002b_31.jpg
en002b_32.jpg
en002b_33.jpg
en002b_34.jpg
en002b_35.jpg
en002b_36.jpg
en002b_37.jpg
en002b_38.jpg
en002b_39.jpg
en002b_4.jpg
en002b_40.jpg
en002b_41.jpg
en002b_42.jpg
en002b_43.jpg
en002b_44.jpg
en002b_45.jpg
en002b_46.jpg
en002b_47.jpg
en002b_48.jpg
en002b_49.jpg
en002b_5.jpg
en002b_50.jpg
en002b_51.jpg
en002b_52.jpg
en002b_53.jpg
en002b_54.jpg
en002b_55.jpg
en002b_56.jpg
en002b_57.jpg
en002b_58.jpg
en002b_59.jpg
en002b_6.jpg
en002b_60.jpg
en002b_61.jpg
en002b_62.jpg
en002b_63.jpg
en002b_64.jpg
en002b_65.jpg
en002b_66.jpg
en002b_67.jpg
en002b_68.jpg
en002b_69.jpg
en002b_7.jpg
en002b_70.jpg
en002b_71.jpg
en002b_72.jpg
en002b_73.jpg
en002b_74.jpg
en002b_75.jpg
en002b_76.jpg
en002b_77.jpg
en002b_78.jpg
en002b_79.jpg
en002b_8.jpg
en002b_80.jpg
en002b_81.jpg
en00

en004b_75.jpg
en004b_76.jpg
en004b_77.jpg
en004b_78.jpg
en004b_79.jpg
en004b_8.jpg
en004b_80.jpg
en004b_81.jpg
en004b_82.jpg
en004b_83.jpg
en004b_84.jpg
en004b_85.jpg
en004b_86.jpg
en004b_87.jpg
en004b_88.jpg
en004b_89.jpg
en004b_9.jpg
en004b_90.jpg
en004b_91.jpg
en004b_92.jpg
en004b_93.jpg
en004b_94.jpg
en004b_95.jpg
en004b_96.jpg
en004b_97.jpg
en004b_98.jpg
en004b_99.jpg
en005a_0.jpg
en005a_1.jpg
en005a_10.jpg
en005a_100.jpg
en005a_101.jpg
en005a_102.jpg
en005a_103.jpg
en005a_104.jpg
en005a_105.jpg
en005a_106.jpg
en005a_107.jpg
en005a_108.jpg
en005a_109.jpg
en005a_11.jpg
en005a_110.jpg
en005a_111.jpg
en005a_112.jpg
en005a_113.jpg
en005a_114.jpg
en005a_115.jpg
en005a_116.jpg
en005a_117.jpg
en005a_118.jpg
en005a_119.jpg
en005a_12.jpg
en005a_120.jpg
en005a_121.jpg
en005a_122.jpg
en005a_123.jpg
en005a_124.jpg
en005a_125.jpg
en005a_126.jpg
en005a_127.jpg
en005a_13.jpg
en005a_14.jpg
en005a_15.jpg
en005a_16.jpg
en005a_17.jpg
en005a_18.jpg
en005a_19.jpg
en005a_2.jpg
en005a_20.jpg
en005a_21.j

en007b_24.jpg
en007b_25.jpg
en007b_26.jpg
en007b_27.jpg
en007b_28.jpg
en007b_29.jpg
en007b_3.jpg
en007b_30.jpg
en007b_31.jpg
en007b_32.jpg
en007b_33.jpg
en007b_34.jpg
en007b_35.jpg
en007b_36.jpg
en007b_37.jpg
en007b_38.jpg
en007b_39.jpg
en007b_4.jpg
en007b_40.jpg
en007b_41.jpg
en007b_42.jpg
en007b_43.jpg
en007b_44.jpg
en007b_45.jpg
en007b_46.jpg
en007b_47.jpg
en007b_48.jpg
en007b_49.jpg
en007b_5.jpg
en007b_50.jpg
en007b_51.jpg
en007b_52.jpg
en007b_53.jpg
en007b_54.jpg
en007b_55.jpg
en007b_56.jpg
en007b_57.jpg
en007b_58.jpg
en007b_59.jpg
en007b_6.jpg
en007b_60.jpg
en007b_61.jpg
en007b_62.jpg
en007b_63.jpg
en007b_64.jpg
en007b_65.jpg
en007b_66.jpg
en007b_67.jpg
en007b_68.jpg
en007b_69.jpg
en007b_7.jpg
en007b_70.jpg
en007b_71.jpg
en007b_72.jpg
en007b_73.jpg
en007b_74.jpg
en007b_75.jpg
en007b_76.jpg
en007b_77.jpg
en007b_78.jpg
en007b_79.jpg
en007b_8.jpg
en007b_80.jpg
en007b_81.jpg
en007b_82.jpg
en007b_83.jpg
en007b_84.jpg
en007b_85.jpg
en007b_86.jpg
en007b_87.jpg
en007b_88.jpg
en007b_89.jp

en009a_113.jpg
en009a_114.jpg
en009a_115.jpg
en009a_116.jpg
en009a_117.jpg
en009a_118.jpg
en009a_12.jpg
en009a_13.jpg
en009a_14.jpg
en009a_15.jpg
en009a_16.jpg
en009a_17.jpg
en009a_18.jpg
en009a_19.jpg
en009a_2.jpg
en009a_20.jpg
en009a_21.jpg
en009a_22.jpg
en009a_23.jpg
en009a_24.jpg
en009a_25.jpg
en009a_26.jpg
en009a_27.jpg
en009a_28.jpg
en009a_29.jpg
en009a_3.jpg
en009a_30.jpg
en009a_31.jpg
en009a_32.jpg
en009a_33.jpg
en009a_34.jpg
en009a_35.jpg
en009a_36.jpg
en009a_37.jpg
en009a_38.jpg
en009a_39.jpg
en009a_4.jpg
en009a_40.jpg
en009a_41.jpg
en009a_42.jpg
en009a_43.jpg
en009a_44.jpg
en009a_45.jpg
en009a_46.jpg
en009a_47.jpg
en009a_48.jpg
en009a_49.jpg
en009a_5.jpg
en009a_50.jpg
en009a_51.jpg
en009a_52.jpg
en009a_53.jpg
en009a_54.jpg
en009a_55.jpg
en009a_56.jpg
en009a_57.jpg
en009a_58.jpg
en009a_59.jpg
en009a_6.jpg
en009a_60.jpg
en009a_61.jpg
en009a_62.jpg
en009a_63.jpg
en009a_64.jpg
en009a_65.jpg
en009a_66.jpg
en009a_67.jpg
en009a_68.jpg
en009a_69.jpg
en009a_7.jpg
en009a_70.jpg
en009a

...

en046a_23.jpg
en046a_24.jpg
en046a_25.jpg
en046a_26.jpg
en046a_27.jpg
en046a_28.jpg
en046a_29.jpg
en046a_3.jpg
en046a_30.jpg
en046a_31.jpg
en046a_32.jpg
en046a_33.jpg
en046a_34.jpg
en046a_35.jpg
en046a_36.jpg
en046a_37.jpg
en046a_38.jpg
en046a_39.jpg
en046a_4.jpg
en046a_40.jpg
en046a_41.jpg
en046a_42.jpg
en046a_43.jpg
en046a_44.jpg
en046a_45.jpg
en046a_46.jpg
en046a_47.jpg
en046a_48.jpg
en046a_49.jpg
en046a_5.jpg
en046a_50.jpg
en046a_51.jpg
en046a_52.jpg
en046a_53.jpg
en046a_54.jpg
en046a_55.jpg
en046a_56.jpg
en046a_57.jpg
en046a_58.jpg
en046a_59.jpg
en046a_6.jpg
en046a_60.jpg
en046a_61.jpg
en046a_62.jpg
en046a_63.jpg
en046a_64.jpg
en046a_65.jpg
en046a_66.jpg
en046a_67.jpg
en046a_68.jpg
en046a_69.jpg
en046a_7.jpg
en046a_70.jpg
en046a_71.jpg
en046a_72.jpg
en046a_73.jpg
en046a_74.jpg
en046a_75.jpg
en046a_76.jpg
en046a_77.jpg
en046a_78.jpg
en046a_79.jpg
en046a_8.jpg
en046a_80.jpg
en046a_81.jpg
en046a_82.jpg
en046a_83.jpg
en046a_84.jpg
en046a_85.jpg
en046a_86.jpg
en046a_87.jpg
en046a_88.jp

en048a_22.jpg
en048a_220.jpg
en048a_221.jpg
en048a_222.jpg
en048a_223.jpg
en048a_224.jpg
en048a_225.jpg
en048a_226.jpg
en048a_227.jpg
en048a_228.jpg
en048a_229.jpg
en048a_23.jpg
en048a_230.jpg
en048a_231.jpg
en048a_232.jpg
en048a_233.jpg
en048a_234.jpg
en048a_235.jpg
en048a_236.jpg
en048a_237.jpg
en048a_238.jpg
en048a_239.jpg
en048a_24.jpg
en048a_240.jpg
en048a_241.jpg
en048a_242.jpg
en048a_243.jpg
en048a_244.jpg
en048a_245.jpg
en048a_246.jpg
en048a_25.jpg
en048a_26.jpg
en048a_27.jpg
en048a_28.jpg
en048a_29.jpg
en048a_3.jpg
en048a_30.jpg
en048a_31.jpg
en048a_32.jpg
en048a_33.jpg
en048a_34.jpg
en048a_35.jpg
en048a_36.jpg
en048a_37.jpg
en048a_38.jpg
en048a_39.jpg
en048a_4.jpg
en048a_40.jpg
en048a_41.jpg
en048a_42.jpg
en048a_43.jpg
en048a_44.jpg
en048a_45.jpg
en048a_46.jpg
en048a_47.jpg
en048a_48.jpg
en048a_49.jpg
en048a_5.jpg
en048a_50.jpg
en048a_51.jpg
en048a_52.jpg
en048a_53.jpg
en048a_54.jpg
en048a_55.jpg
en048a_56.jpg
en048a_57.jpg
en048a_58.jpg
en048a_59.jpg
en048a_6.jpg
en048a_60.j

en049b_4.jpg
en049b_40.jpg
en049b_41.jpg
en049b_42.jpg
en049b_43.jpg
en049b_44.jpg
en049b_45.jpg
en049b_46.jpg
en049b_47.jpg
en049b_48.jpg
en049b_49.jpg
en049b_5.jpg
en049b_50.jpg
en049b_51.jpg
en049b_52.jpg
en049b_53.jpg
en049b_54.jpg
en049b_55.jpg
en049b_56.jpg
en049b_57.jpg
en049b_58.jpg
en049b_59.jpg
en049b_6.jpg
en049b_60.jpg
en049b_61.jpg
en049b_62.jpg
en049b_63.jpg
en049b_64.jpg
en049b_65.jpg
en049b_66.jpg
en049b_67.jpg
en049b_68.jpg
en049b_69.jpg
en049b_7.jpg
en049b_70.jpg
en049b_71.jpg
en049b_72.jpg
en049b_73.jpg
en049b_74.jpg
en049b_75.jpg
en049b_76.jpg
en049b_77.jpg
en049b_78.jpg
en049b_79.jpg
en049b_8.jpg
en049b_80.jpg
en049b_81.jpg
en049b_82.jpg
en049b_83.jpg
en049b_84.jpg
en049b_85.jpg
en049b_86.jpg
en049b_87.jpg
en049b_88.jpg
en049b_89.jpg
en049b_9.jpg
en049b_90.jpg
en049b_91.jpg
en049b_92.jpg
en049b_93.jpg
en049b_94.jpg
en049b_95.jpg
en049b_96.jpg
en049b_97.jpg
en049b_98.jpg
en049b_99.jpg
en050a_0.jpg
en050a_1.jpg
en050a_10.jpg
en050a_100.jpg
en050a_101.jpg
en050a_102.j

C:\Users\yimen\Anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


123/123 [==============================] - ETA: 0s - loss: 0.0862 - accuracy: 0.0779
Epoch 2: val_loss improved from 402.12119 to 336.48962, saving model to cnn_model\newckpt.h5
123/123 [==============================] - 56s 452ms/step - loss: 0.0862 - accuracy: 0.0779 - val_loss: 336.4896 - val_accuracy: 0.0000e+00
Epoch 3/100
123/123 [==============================] - ETA: 0s - loss: 0.0912 - accuracy: 0.1814
Epoch 3: val_loss improved from 336.48962 to 157.96033, saving model to cnn_model\newckpt.h5
123/123 [==============================] - 54s 439ms/step - loss: 0.0912 - accuracy: 0.1814 - val_loss: 157.9603 - val_accuracy: 0.0000e+00
Epoch 4/100
123/123 [==============================] - ETA: 0s - loss: 0.0413 - accuracy: 0.4618
Epoch 4: val_loss did not improve from 157.96033
123/123 [==============================] - 52s 424ms/step - loss: 0.0413 - accuracy: 0.4618 - val_loss: 204.1086 - val_accuracy: 0.0000e+00
Epoch 5/100
123/123 [==============================] - ETA: 0s - l

# noise reducer

In [179]:
from scipy.io import wavfile
import noisereduce as nr
# load data

def reduce_noise(input_dir, output_dir):
    for filename in os.listdir(input_dir):
        rate, data = wavfile.read(f"sample_wav_files/{filename}")
        orig_shape = data.shape
        data = np.reshape(data, (2, -1))

        # perform noise reduction
        reduced_noise = nr.reduce_noise(
            y=data,
            sr=rate,
            stationary=True
        )

        wavfile.write(os.path.join(output_dir,filename), rate, reduced_noise.reshape(orig_shape))

In [180]:
reduce_noise('sample_wav_files', 'reduced_noise_wav_files')

<ipython-input-179-0a56794b55f6>:7: WavFileWarning: Chunk (non-data) not understood, skipping it.
  rate, data = wavfile.read(f"sample_wav_files/{filename}")


# SPLIT WAV FILES

In [26]:
import librosa
import numpy as np
import os
import soundfile as sf

def split_wav_librosa(wav_file, target_dir, target_segment_len=1):
    # Load the audio file
    audio, sr = librosa.load(wav_file, sr=None)
    song_name = wav_file.split("/")[-1][:-4]

    # Calculate the number of samples per target segment
    samples_per_segment = int(target_segment_len * sr)

    # Calculate the total number of segments
    total_segments = len(audio) // samples_per_segment

    for i in range(total_segments):
        start_sample = i * samples_per_segment
        end_sample = (i + 1) * samples_per_segment

        # Extract the segment
        segment = audio[start_sample:end_sample]

        # Create a new file for each segment
        segment_filename = os.path.join(target_dir, f"{song_name}_{i + 1}.wav")
#         librosa.output.write_wav(segment_filename, segment, sr)
        sf.write(segment_filename, segment, 48000, 'PCM_24')
        
        
wav_file = "sample_wav_files/en019a.wav"
target_directory = "predict_split_wav"
split_wav_librosa(wav_file, target_directory, target_segment_len=1)

In [49]:
def convert_wav_to_spectrogram(spec_dir, wav_dir):
    specs = [x for x in listdir(spec_dir) if x.endswith('.jpg')]
    wavs = [x for x in listdir(wav_dir) if x.endswith('.wav')]

    for wav in wavs:
        print(wav)
        name = wav[:-4]
        if name + '.jpg' not in specs:
            wav_to_spectrogram(join(wav_dir, name + '.wav'), join(spec_dir, name + '.jpg'))
        print('Done!')

In [50]:
convert_wav_to_spectrogram('split_wav_based_on_midi_spectrogram', 'split_wav_based_on_midi')

en001a_0.wav
Done!
en001a_1.wav
Done!
en001a_10.wav


<ipython-input-5-6076b732de78>:10: UserWarning: amplitude_to_db was called on complex input so phase information will be discarded. To suppress this warning, call amplitude_to_db(np.abs(S)) instead.
  librosa.display.specshow(librosa.amplitude_to_db(C, ref=np.max),
C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=156
  warnings.warn(
C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=207
  warnings.warn(


Done!
en001a_100.wav
Done!
en001a_101.wav
Done!
en001a_102.wav
Done!
en001a_11.wav


C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=194
  warnings.warn(
C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=220
  warnings.warn(
C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=110
  warnings.warn(
C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=201
  warnings.warn(
C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=162
  warnings.warn(


Done!
en001a_12.wav
Done!
en001a_13.wav
Done!
en001a_14.wav
Done!
en001a_15.wav


...

Done!
en045a_97.wav
Done!
en045a_98.wav
Done!
en045a_99.wav
Done!
en045b_0.wav
Done!
en045b_1.wav
Done!
en045b_10.wav
Done!
en045b_100.wav
Done!
en045b_101.wav
Done!
en045b_102.wav
Done!
en045b_103.wav
Done!
en045b_104.wav
Done!
en045b_105.wav
Done!
en045b_106.wav
Done!
en045b_107.wav
Done!
en045b_108.wav
Done!
en045b_109.wav
Done!
en045b_11.wav
Done!
en045b_110.wav
Done!
en045b_111.wav
Done!
en045b_112.wav
Done!
en045b_113.wav
Done!
en045b_114.wav
Done!
en045b_115.wav
Done!
en045b_12.wav
Done!
en045b_13.wav
Done!
en045b_14.wav
Done!
en045b_15.wav
Done!
en045b_16.wav
Done!
en045b_17.wav
Done!
en045b_18.wav
Done!
en045b_19.wav
Done!
en045b_2.wav
Done!
en045b_20.wav
Done!
en045b_21.wav
Done!
en045b_22.wav
Done!
en045b_23.wav
Done!
en045b_24.wav
Done!
en045b_25.wav
Done!
en045b_26.wav
Done!
en045b_27.wav
Done!
en045b_28.wav
Done!
en045b_29.wav
Done!
en045b_3.wav
Done!
en045b_30.wav
Done!
en045b_31.wav
Done!
en045b_32.wav
Done!
en045b_33.wav
Done!
en045b_34.wav
Done!
en045b_35.wav
Done!
en

Done!
en047a_37.wav
Done!
en047a_38.wav
Done!
en047a_39.wav
Done!
en047a_4.wav
Done!
en047a_40.wav
Done!
en047a_41.wav
Done!
en047a_42.wav
Done!
en047a_43.wav
Done!
en047a_44.wav
Done!
en047a_45.wav
Done!
en047a_46.wav
Done!
en047a_47.wav
Done!
en047a_48.wav
Done!
en047a_49.wav
Done!
en047a_5.wav
Done!
en047a_50.wav
Done!
en047a_51.wav
Done!
en047a_52.wav
Done!
en047a_53.wav
Done!
en047a_54.wav
Done!
en047a_55.wav
Done!
en047a_56.wav
Done!
en047a_57.wav
Done!
en047a_58.wav
Done!
en047a_59.wav
Done!
en047a_6.wav
Done!
en047a_60.wav
Done!
en047a_61.wav
Done!
en047a_62.wav
Done!
en047a_63.wav
Done!
en047a_64.wav
Done!
en047a_65.wav
Done!
en047a_66.wav
Done!
en047a_67.wav
Done!
en047a_68.wav
Done!
en047a_69.wav
Done!
en047a_7.wav
Done!
en047a_70.wav
Done!
en047a_71.wav
Done!
en047a_72.wav
Done!
en047a_73.wav
Done!
en047a_74.wav
Done!
en047a_75.wav
Done!
en047a_76.wav
Done!
en047a_77.wav
Done!
en047a_78.wav
Done!
en047a_79.wav
Done!
en047a_8.wav
Done!
en047a_80.wav
Done!
en047a_81.wav
Done!

Done!
en048a_32.wav
Done!
en048a_33.wav
Done!
en048a_34.wav
Done!
en048a_35.wav
Done!
en048a_36.wav
Done!
en048a_37.wav
Done!
en048a_38.wav
Done!
en048a_39.wav
Done!
en048a_4.wav
Done!
en048a_40.wav
Done!
en048a_41.wav
Done!
en048a_42.wav
Done!
en048a_43.wav
Done!
en048a_44.wav
Done!
en048a_45.wav
Done!
en048a_46.wav
Done!
en048a_47.wav
Done!
en048a_48.wav
Done!
en048a_49.wav
Done!
en048a_5.wav
Done!
en048a_50.wav
Done!
en048a_51.wav
Done!
en048a_52.wav
Done!
en048a_53.wav
Done!
en048a_54.wav
Done!
en048a_55.wav
Done!
en048a_56.wav
Done!
en048a_57.wav
Done!
en048a_58.wav
Done!
en048a_59.wav
Done!
en048a_6.wav
Done!
en048a_60.wav
Done!
en048a_61.wav
Done!
en048a_62.wav
Done!
en048a_63.wav
Done!
en048a_64.wav
Done!
en048a_65.wav
Done!
en048a_66.wav
Done!
en048a_67.wav
Done!
en048a_68.wav
Done!
en048a_69.wav
Done!
en048a_7.wav
Done!
en048a_70.wav
Done!
en048a_71.wav
Done!
en048a_72.wav
Done!
en048a_73.wav
Done!
en048a_74.wav
Done!
en048a_75.wav
Done!
en048a_76.wav
Done!
en048a_77.wav
Done

Done!
en049a_45.wav
Done!
en049a_46.wav
Done!
en049a_47.wav
Done!
en049a_48.wav
Done!
en049a_49.wav
Done!
en049a_5.wav
Done!
en049a_50.wav
Done!
en049a_51.wav
Done!
en049a_52.wav
Done!
en049a_53.wav
Done!
en049a_54.wav
Done!
en049a_55.wav
Done!
en049a_56.wav
Done!
en049a_57.wav
Done!
en049a_58.wav
Done!
en049a_59.wav
Done!
en049a_6.wav
Done!
en049a_60.wav
Done!
en049a_61.wav
Done!
en049a_62.wav
Done!
en049a_63.wav
Done!
en049a_64.wav
Done!
en049a_65.wav
Done!
en049a_66.wav
Done!
en049a_67.wav
Done!
en049a_68.wav
Done!
en049a_69.wav
Done!
en049a_7.wav
Done!
en049a_70.wav
Done!
en049a_71.wav
Done!
en049a_72.wav
Done!
en049a_73.wav
Done!
en049a_74.wav
Done!
en049a_75.wav
Done!
en049a_76.wav
Done!
en049a_77.wav
Done!
en049a_78.wav
Done!
en049a_79.wav
Done!
en049a_8.wav
Done!
en049a_80.wav
Done!
en049a_81.wav
Done!
en049a_82.wav
Done!
en049a_83.wav
Done!
en049a_84.wav
Done!
en049a_85.wav
Done!
en049a_86.wav
Done!
en049a_87.wav
Done!
en049a_88.wav
Done!
en049a_89.wav
Done!
en049a_9.wav
Done!

Done!
en050b_124.wav
Done!
en050b_125.wav
Done!
en050b_126.wav
Done!
en050b_127.wav
Done!
en050b_128.wav
Done!
en050b_129.wav
Done!
en050b_13.wav
Done!
en050b_130.wav
Done!
en050b_131.wav
Done!
en050b_132.wav
Done!
en050b_133.wav
Done!
en050b_134.wav
Done!
en050b_135.wav
Done!
en050b_136.wav
Done!
en050b_137.wav
Done!
en050b_138.wav
Done!
en050b_139.wav
Done!
en050b_14.wav
Done!
en050b_140.wav
Done!
en050b_141.wav
Done!
en050b_142.wav
Done!
en050b_143.wav
Done!
en050b_144.wav
Done!
en050b_145.wav
Done!
en050b_146.wav
Done!
en050b_147.wav
Done!
en050b_148.wav
Done!
en050b_149.wav
Done!
en050b_15.wav
Done!
en050b_150.wav
Done!
en050b_151.wav
Done!
en050b_152.wav
Done!
en050b_153.wav
Done!
en050b_154.wav
Done!
en050b_155.wav
Done!
en050b_156.wav
Done!
en050b_157.wav
Done!
en050b_158.wav
Done!
en050b_159.wav
Done!
en050b_16.wav
Done!
en050b_160.wav
Done!
en050b_161.wav
Done!
en050b_162.wav
Done!
en050b_163.wav
Done!
en050b_164.wav
Done!
en050b_165.wav
Done!
en050b_166.wav
Done!
en050b_167.

In [88]:
convert_wav_to_spectrogram('predict_spectrogram', 'predict_split_wav')

en019a_1.wav
Done!
en019a_10.wav
Done!
en019a_11.wav
Done!
en019a_12.wav
Done!
en019a_13.wav
Done!
en019a_14.wav
Done!
en019a_15.wav
Done!
en019a_16.wav
Done!
en019a_17.wav
Done!
en019a_18.wav
Done!
en019a_19.wav
Done!
en019a_2.wav
Done!
en019a_20.wav
Done!
en019a_21.wav
Done!
en019a_22.wav
Done!
en019a_23.wav
Done!
en019a_24.wav
Done!
en019a_25.wav
Done!
en019a_26.wav
Done!
en019a_27.wav
Done!
en019a_28.wav
Done!
en019a_29.wav
Done!
en019a_3.wav
Done!
en019a_30.wav
Done!
en019a_31.wav
Done!
en019a_32.wav
Done!
en019a_33.wav
Done!
en019a_34.wav
Done!
en019a_35.wav
Done!
en019a_36.wav
Done!
en019a_37.wav
Done!
en019a_38.wav
Done!
en019a_39.wav
Done!
en019a_4.wav
Done!
en019a_40.wav
Done!
en019a_41.wav
Done!
en019a_42.wav
Done!
en019a_43.wav
Done!
en019a_44.wav
Done!
en019a_45.wav
Done!
en019a_46.wav
Done!
en019a_47.wav
Done!
en019a_48.wav
Done!
en019a_49.wav
Done!
en019a_5.wav
Done!
en019a_50.wav
Done!
en019a_51.wav
Done!
en019a_52.wav
Done!
en019a_53.wav
Done!
en019a_54.wav
Done!
en019

In [166]:
convert_wav_to_spectrogram('predict_spectrogram', 'predict_split_wav_by_silence')

segment_1.wav
Done!
segment_10.wav
Done!
segment_11.wav
Done!
segment_12.wav
Done!
segment_13.wav
Done!
segment_14.wav
Done!
segment_15.wav
Done!
segment_16.wav
Done!
segment_17.wav
Done!
segment_18.wav
Done!
segment_19.wav
Done!
segment_2.wav
Done!
segment_20.wav
Done!
segment_21.wav
Done!
segment_22.wav
Done!
segment_23.wav
Done!
segment_24.wav
Done!
segment_25.wav
Done!
segment_26.wav
Done!
segment_27.wav
Done!
segment_28.wav
Done!
segment_29.wav
Done!
segment_3.wav
Done!
segment_30.wav
Done!
segment_31.wav
Done!
segment_32.wav
Done!
segment_33.wav
Done!
segment_34.wav
Done!
segment_35.wav
Done!
segment_36.wav
Done!
segment_37.wav
Done!
segment_38.wav
Done!
segment_39.wav
Done!
segment_4.wav
Done!
segment_40.wav
Done!
segment_41.wav
Done!
segment_42.wav
Done!
segment_43.wav
Done!
segment_44.wav
Done!
segment_45.wav
Done!
segment_46.wav
Done!
segment_47.wav
Done!
segment_48.wav
Done!
segment_49.wav
Done!
segment_5.wav
Done!
segment_50.wav
Done!
segment_6.wav
Done!
segment_7.wav
Done!

In [17]:
import librosa
import numpy as np
import soundfile as sf

def displayTime(startFrame, endFrame, sr):    
    print(' start time: ' + str(startFrame/sr) + ', end time: ' + str(endFrame/sr))

# on silence 
def split_and_save_wav(input_wav, output_folder, silence_threshold=10, min_segment_duration=0.5):
    # Load the input WAV file
    y, sr = librosa.load(input_wav, sr=None)
    print(sr)
    # Compute the envelope of the audio signal
    envelope = np.abs(librosa.effects.preemphasis(y))

    # Detect non-silent intervals based on a silence threshold
    non_silent_intervals = librosa.effects.split(y, top_db=silence_threshold)
    print(non_silent_intervals)
    for i in non_silent_intervals:    
        displayTime(i[0],i[1], sr)
    # Create a list to store the segments
    segments = []
    print('done')
    # Iterate over non-silent intervals and extract segments
    for i, (start, end) in enumerate(non_silent_intervals):
        start_idx = int(start)
        end_idx = int(end)
        duration = end - start

        # Segment from time 0 to the current end time
        if i == 0:
            segments.append(y[0:end_idx])

        # Segment from the previous end time to the current end time
        if duration >= min_segment_duration:
            prev_end_idx = int(non_silent_intervals[i - 1][1]) if i > 0 else 0
            displayTime(prev_end_idx,end_idx, sr)
            segments.append(y[prev_end_idx:end_idx])

    # Save each segment as a separate WAV file
    for i, segment in enumerate(segments):
        output_file = f"{output_folder}/segment_{i + 1}.wav"
        print(output_file)
        sf.write(output_file, segment, sr)






In [18]:
# Split and save the WAV file into segments based on silence
input_wav_file = "sample_wav_files/en001a.wav"
output_folder = "split_wav_by_silence"
split_and_save_wav(input_wav_file, output_folder, silence_threshold=10, min_segment_duration=1)

44100
[[ 158720  178688]
 [ 185856  202752]
 [ 211968  230400]
 [ 239616  249344]
 [ 265216  282112]
 [ 315392  325120]
 [ 344064  357376]
 [ 369664  382976]
 [ 397824  404480]
 [ 422400  431616]
 [ 436224  439296]
 [ 452096  453632]
 [ 529408  545280]
 [ 554496  571392]
 [ 582144  599040]
 [ 635904  679424]
 [ 689664  690176]
 [ 740864  750592]
 [ 753664  783360]
 [ 792576  801792]
 [ 845824  865792]
 [ 872448  888832]
 [ 900608  910848]
 [ 954368  969216]
 [ 980992  990720]
 [1005056 1025024]
 [1031168 1050112]
 [1058816 1077248]
 [1083392 1099776]
 [1110528 1122816]
 [1163264 1172480]
 [1191424 1206784]
 [1245184 1259008]
 [1296896 1307648]
 [1642496 1662976]
 [1668096 1688064]
 [1696768 1712640]
 [1722368 1733632]
 [1747456 1763840]
 [1800704 1813504]
 [1829376 1844224]
 [1853952 1870336]
 [1881600 1891840]
 [1907712 1915904]
 [1920000 1923072]
 [2012672 2026496]
 [2037760 2055680]
 [2065408 2083328]
 [2119168 2162176]
 [2176000 2184192]
 [2223104 2234368]
 [2236416 2268160]
 [2275

split_wav_by_silence/segment_13.wav
split_wav_by_silence/segment_14.wav
split_wav_by_silence/segment_15.wav
split_wav_by_silence/segment_16.wav
split_wav_by_silence/segment_17.wav
split_wav_by_silence/segment_18.wav
split_wav_by_silence/segment_19.wav
split_wav_by_silence/segment_20.wav
split_wav_by_silence/segment_21.wav
split_wav_by_silence/segment_22.wav
split_wav_by_silence/segment_23.wav
split_wav_by_silence/segment_24.wav
split_wav_by_silence/segment_25.wav
split_wav_by_silence/segment_26.wav
split_wav_by_silence/segment_27.wav
split_wav_by_silence/segment_28.wav
split_wav_by_silence/segment_29.wav
split_wav_by_silence/segment_30.wav
split_wav_by_silence/segment_31.wav
split_wav_by_silence/segment_32.wav
split_wav_by_silence/segment_33.wav
split_wav_by_silence/segment_34.wav
split_wav_by_silence/segment_35.wav
split_wav_by_silence/segment_36.wav
split_wav_by_silence/segment_37.wav
split_wav_by_silence/segment_38.wav
split_wav_by_silence/segment_39.wav
split_wav_by_silence/segment

In [27]:
convert_wav_to_spectrogram("split_wav_by_silence_to_spectrogram", "split_wav_by_silence")

segment_1.wav
Done!
segment_10.wav
Done!
segment_11.wav
Done!
segment_12.wav


<ipython-input-5-6076b732de78>:10: UserWarning: amplitude_to_db was called on complex input so phase information will be discarded. To suppress this warning, call amplitude_to_db(np.abs(S)) instead.
  librosa.display.specshow(librosa.amplitude_to_db(C, ref=np.max),
C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=168
  warnings.warn(
C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=212
  warnings.warn(
C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=240
  warnings.warn(
C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=120
  warnings.warn(
C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too lar

Done!
segment_13.wav
Done!
segment_14.wav
Done!
segment_15.wav
Done!
segment_16.wav
Done!
segment_17.wav


C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=224
  warnings.warn(
C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=112
  warnings.warn(
C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=204
  warnings.warn(
C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=216
  warnings.warn(


Done!
segment_18.wav
Done!
segment_19.wav
Done!
segment_2.wav
Done!
segment_20.wav
Done!
segment_21.wav


...

Done!
segment_60.wav
Done!
segment_61.wav
Done!
segment_62.wav
Done!
segment_63.wav
Done!
segment_64.wav
Done!
segment_65.wav
Done!
segment_66.wav
Done!
segment_67.wav
Done!
segment_68.wav
Done!
segment_69.wav


C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=232
  warnings.warn(
C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=116
  warnings.warn(
C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=108
  warnings.warn(


Done!
segment_7.wav
Done!
segment_70.wav
Done!
segment_71.wav
Done!
segment_8.wav


C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=88
  warnings.warn(
C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=44
  warnings.warn(
C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=252
  warnings.warn(
C:\Users\yimen\Anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=256 is too large for input signal of length=200
  warnings.warn(


Done!
segment_9.wav
Done!


# PREDICT MIDI FROM WAV

In [56]:
import numpy as np
import keras
from keras.layers import Dense, Flatten, Reshape, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger
from keras.models import load_model

from  skimage.measure import block_reduce
from PIL import Image
import pretty_midi as pm 
import os, os.path
import re

model_path = "cnn_model/newckpt.h5"
image_path = "predict_spectrogram"
#image_file = "daylight_41.jpg"

# midi_path = "daylight_41.mid"
# midi = pm.PrettyMIDI(midi_path)

# one_hot = utils.pretty_midi_to_one_hot(midi)
# np.savetxt("onehot_short.csv",one_hot, delimiter=",")
# y = utils.slice_to_categories(one_hot)

x = []

model = load_model(model_path)
model.compile(loss=keras.losses.binary_crossentropy,
            optimizer=keras.optimizers.Adam(),
            metrics=['accuracy'])

filenums = []
for image_file in os.listdir(image_path):
	print(image_file)
	
	im = Image.open(os.path.join(image_path, image_file))
	im = im.crop((14, 13, 594, 301))
	resize = im.resize((49, 145), Image.NEAREST)
	resize.load()
	arr = np.asarray(resize, dtype="float32")

	x.append(arr)
	filenums.append(int(re.search(r'\d+', image_file).group()))

x = np.array(x)
x /= 255.0

y_pred = model.predict(x)
# print(np.nonzero(y))
print(y_pred)

notes_unsorted = [np.argmax(y_pred[n]) for n in range(len(y_pred))]

notes = [x for _,x in sorted(zip(filenums, notes_unsorted))]
print(notes)

i=0
for note in notes:
	one_hot = np.zeros((128, 25))
	one_hot[note, :] = 1
	mid = one_hot_to_pretty_midi(one_hot)
	mid.write('predicted_output_midi_split/test_' + str(i) + ".mid")
	i += 1

segment_1.jpg
segment_10.jpg
segment_11.jpg
segment_12.jpg
segment_13.jpg
segment_14.jpg
segment_15.jpg
segment_16.jpg
segment_17.jpg
segment_18.jpg
segment_19.jpg
segment_2.jpg
segment_20.jpg
segment_21.jpg
segment_22.jpg
segment_23.jpg
segment_24.jpg
segment_25.jpg
segment_26.jpg
segment_27.jpg
segment_28.jpg
segment_29.jpg
segment_3.jpg
segment_30.jpg
segment_31.jpg
segment_32.jpg
segment_33.jpg
segment_34.jpg
segment_35.jpg
segment_36.jpg
segment_37.jpg
segment_38.jpg
segment_39.jpg
segment_4.jpg
segment_40.jpg
segment_41.jpg
segment_42.jpg
segment_43.jpg
segment_44.jpg
segment_45.jpg
segment_46.jpg
segment_47.jpg
segment_48.jpg
segment_49.jpg
segment_5.jpg
segment_50.jpg
segment_51.jpg
segment_52.jpg
segment_53.jpg
segment_54.jpg
segment_55.jpg
segment_56.jpg
segment_57.jpg
segment_58.jpg
segment_59.jpg
segment_6.jpg
segment_60.jpg
segment_61.jpg
segment_62.jpg
segment_63.jpg
segment_64.jpg
segment_65.jpg
segment_66.jpg
segment_67.jpg
segment_68.jpg
segment_69.jpg
segment_7.jpg
se

In [168]:
split_midi_files('predicted_output_midi_split','', 'predicted_merge_output_midi/test_output.mid', 1, merge=True)

['predicted_output_midi_split\\test_0.mid', 'predicted_output_midi_split\\test_1.mid', 'predicted_output_midi_split\\test_2.mid', 'predicted_output_midi_split\\test_3.mid', 'predicted_output_midi_split\\test_4.mid', 'predicted_output_midi_split\\test_5.mid', 'predicted_output_midi_split\\test_6.mid', 'predicted_output_midi_split\\test_7.mid', 'predicted_output_midi_split\\test_8.mid', 'predicted_output_midi_split\\test_9.mid', 'predicted_output_midi_split\\test_10.mid', 'predicted_output_midi_split\\test_11.mid', 'predicted_output_midi_split\\test_12.mid', 'predicted_output_midi_split\\test_13.mid', 'predicted_output_midi_split\\test_14.mid', 'predicted_output_midi_split\\test_15.mid', 'predicted_output_midi_split\\test_16.mid', 'predicted_output_midi_split\\test_17.mid', 'predicted_output_midi_split\\test_18.mid', 'predicted_output_midi_split\\test_19.mid', 'predicted_output_midi_split\\test_20.mid', 'predicted_output_midi_split\\test_21.mid', 'predicted_output_midi_split\\test_22.mid

AttributeError: 'int' object has no attribute 'time'